<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка</a></span><ul class="toc-item"><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Предобработка текста</a></span><ul class="toc-item"><li><span><a href="#Лемматизируем-текст" data-toc-modified-id="Лемматизируем-текст-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Лемматизируем текст</a></span></li><li><span><a href="#Уберем-неинформативные-символы-из-текста" data-toc-modified-id="Уберем-неинформативные-символы-из-текста-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Уберем неинформативные символы из текста</a></span></li><li><span><a href="#Закодируем-чистый-текс-в-векторы" data-toc-modified-id="Закодируем-чистый-текс-в-векторы-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Закодируем чистый текс в векторы</a></span></li></ul></li><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Выводы:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель CatBoost</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Введение

    Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
    
    В нашем распоряжении набор данных с разметкой о токсичности правок.
    Наша задача - построить модель со значением метрики качества F1 не меньше 0.75. 


## Предобработка

In [6]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\krulo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krulo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krulo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\krulo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [7]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Предобработка текста


#### Лемматизируем текст

In [8]:
# from nltk.corpus import wordnet
# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)

In [9]:

# def lemmatize(text):
#     word_list = nltk.word_tokenize(text)
#     lemmatizer = WordNetLemmatizer()
   
#     return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])

In [11]:

# df['lemm_text'] = df['text'].progress_apply(lemmatize)
# df = df.drop(['text'], axis=1)
# df.head()

#### Уберем неинформативные символы из текста

In [13]:
# напишем функцию для отчистки текста и создадим новый столбец

def clear_text(text):
    text_1 = re.sub(r'[^a-zA-Z ]', ' ', text)
    final_text = " ".join(text_1.split())
    return final_text  

In [14]:
df['clear_text'] = df['text'].apply(clear_text)
df = df.drop(['text'], axis=1)
df.head()

,toxic,clear_text
0,0,Explanation Why the edits made under my userna...
1,0,D aww He matches this background colour I m se...
2,0,Hey man I m really not trying to edit war It s...
3,0,More I can t make any real suggestions on impr...
4,0,You sir are my hero Any chance you remember wh...


In [28]:
df['clear_text'] = df['clear_text'].str.lower()
df.head()

,toxic,clear_text
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...


#### Закодируем чистый текс в векторы

In [16]:
# Разделим выборку на тренировочную и тестовую 
features = df['clear_text']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=42, test_size=0.2, stratify=target)

# проверим по форме
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(127433,)
(31859,)
(127433,)
(31859,)


In [17]:
corpus = features_train.values
corpus_test = features_test.values

In [18]:
# Сформируем список стоп-слов
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krulo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
count_tf = TfidfVectorizer(stop_words = stopwords)

In [20]:
# получаем векторные признаки для обучения
features_train = count_tf.fit_transform(corpus)


In [21]:
# получаем векторные признаки для теста
features_test = count_tf.transform(corpus_test)

In [22]:
# проверим по форме
print(features_train.shape)
print(features_test.shape)

(127433, 147534)
(31859, 147534)


### Выводы:
    Мы убрали из текста ненужные символы, убрали стоп слова и сформировали векторные признаки. Mожем переходить к моделированию.

## Обучение

### Модель логистической регрессии

In [25]:
model_lr = LogisticRegression(random_state=1, class_weight = 'balanced')

params = {'C': np.logspace(-4, 4, 50),
          'penalty': ['l1', 'l2']}

grid_CV = GridSearchCV(estimator=model_lr, 
                         param_grid=params, 
                         scoring = 'f1',
                         cv=5,
                         n_jobs=-1)

grid_CV.fit(features_train, target_train)

C:\Users\krulo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
250 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
250 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\krulo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\krulo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\krulo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
V

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=1),
             n_jobs=-1,
             param_grid={'C': array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095476e-04, 1.38949549e-03,
       2.02358965e-03, 2.94705170e-03, 4.29193426e-03, 6.25055193e-03,
       9.10298178e-03, 1.32571137e-02, 1.93069773e-02, 2.811...
       3.72759372e+00, 5.42867544e+00, 7.90604321e+00, 1.15139540e+01,
       1.67683294e+01, 2.44205309e+01, 3.55648031e+01, 5.17947468e+01,
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04]),
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [26]:
best_params = grid_CV.best_params_
print(best_params)
print()
best_score = grid_CV.best_score_
print('Оценка лучшей модели логистической регрессии на кросс-валидации', best_score)

{'C': 5.428675439323859, 'penalty': 'l2'}

Оценка лучшей модели логистической регрессии на кросс-валидации 0.7692907645331523


<div class="alert alert-info"> 
   class_weight = 'balanced' дает улучшение в третьем знаке после запятой)
</div>

### Модель случайного леса

In [ ]:
# model_rf = RandomForestClassifier(random_state=1)

# params = {'max_depth' : [4,6,8,10],
#           'n_estimators' : range(10,20,2)}

# grid_CV_rf = GridSearchCV(estimator=model_rf, 
#                          param_grid=params, 
#                          scoring = 'f1',
#                          cv=3,
#                          n_jobs=-1)

# grid_CV_rf.fit(features_train, target_train)

In [ ]:
# best_params = grid_CV_rf.best_params_
# print(best_params)
# print()
# best_score = grid_CV_rf.best_score_
# print('Оценка лучшей модели случайного леса на кросс-валидации', best_score)

### Модель CatBoost

In [ ]:
# Добавим валидационные выборки для катбуста

features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, random_state=42, test_size=0.2)

In [ ]:
model_gb = CatBoostClassifier(iterations = 1000, learning_rate = 0.03)

model_gb.fit(features_train, target_train, eval_set=(features_valid, target_valid), verbose=False, use_best_model = True)

print('Params')
print(model_gb.get_params())

In [ ]:
from sklearn.metrics import f1_score
pred = model_gb.predict(features_test)
print('F1 на модели градиентного бустинга', f1_score(target_test, pred))

### Тестирование


In [27]:
# Лучшую оценку на тренировочной выборке получила модель логистической регрессии

preds = grid_CV.predict(features_test)
print('F1 на модели логистической регрессии', f1_score(target_test, preds))

F1 на модели логистической регрессии 0.7675457558726041


## Выводы

    В этом проекте нам была поставлена задача определения токсичности твитов. Для работы у нас было более 160 тысяч размеченных сообщений. 
   
    В ходе подготовки данных к обучению мы 
        1. лемматизировали (нет) данные
        2. избавились от слов, которые не несут содержательного значения (стоп-слова)
        3. перевели сообщения в векторный формат
       
    На этапе обучения мы рассмотрели три модели: логистическую регрессию, случайный лес и КатБуст. Лучший результат показала модель логистической регрессии. На тестовой выборке мы молучили значение метрики F1 на уровне 76%, что превышало необходимое значение в техническом задании.
    
    Достижение необходимой точности предсказания модели стало возможным благодаря перебору гиперпараметров моделей с помощью кросс-валидации. 